In [3]:
from scipy import integrate
import numpy as np
import math as m
from scipy.interpolate import interp1d
from scipy import integrate
import matplotlib.pyplot as plt
from ase.units import Bohr, Hartree

r=np.load('MoS2_alone_r.npy')
rx=r
ry=r
#W_ee=np.load('MoS2_alone_W_ee.npy')
W_eh=np.load('MoS2_alone_W_eh.npy')
W_ee=W_eh

$$
\Psi=\frac{1}{\sqrt{2}}\frac{2}{\pi a b Norm}(e^{-x/a}e^{-y/b}+e^{-y/a}e^{-x/b})
$$

$$
Norm=\sqrt{1+\frac{16a^2b^2}{(a+b)^4}}
$$

In [4]:
from scipy.integrate import quad

def wavefunc(x,y,aa,bb):
    Norm=np.sqrt(1+16*aa**2*bb**2/(aa+bb)**4)
    return 1/np.sqrt(2)/Norm*(2*np.exp(-x/aa)/(aa**2*np.pi)*np.exp(-y/bb)/(bb**2)+
                      2*np.exp(-y/aa)/(aa**2*np.pi)*np.exp(-x/bb)/(bb**2))

In [5]:
Wee = interp1d(r,W_ee,kind='linear')
#plt.plot(r,Wee(r))
#plt.plot(r,W_ee,'k.')

In [6]:
Weh = interp1d(r,W_eh,kind='linear')
#plt.plot(r,Weh(r))
#plt.plot(r,W_eh,'k.')

$$
V_{1}=(2\pi)^2 \int_0^\infty\int_0^\infty{\Psi (r_{1}, r_{2})^2 W_{eh}(r_{1})r_{1} r_{2}dr_{1}dr_{2}}
$$

In [7]:
def integrand11(rx,ry,a,b):
    return 27.21*rx*ry*wavefunc(rx,ry,a,b)**2*Weh(rx)

In [8]:
def V11(ry,aa,bb):    
    return 2*np.pi*integrate.quad(integrand11, rx[0], rx[-1],
                                    args=(ry,aa,bb))[0]

In [9]:
def V22(aa,bb):    
    return 2*np.pi*integrate.quad(V11, ry[0], ry[-1],
                                    args=(aa,bb))[0]

$$
V_{2}=(2\pi)^2 \int_0^\infty\int_0^\infty{\Psi (r_{1}, r_{2})^2 W_{eh}(|r_{1}-r_{2}|)r_{1} r_{2}dr_{1}dr_{2}}
$$

In [10]:
def integrand22(rx,ry,a,b):
    if abs(rx-ry)<0.1: return 0
    else:
        return 27.21*rx*wavefunc(rx,ry,a,b)**2*Wee(abs(ry-rx))

In [11]:
def V33(ry,aa,bb):    
    return 2*np.pi*ry*integrate.quad(integrand22, rx[0], rx[-1],
                                    args=(ry,aa,bb), limit=200)[0]

In [12]:
def V44(aa,bb):    
    return 2*np.pi*integrate.quad(V33, ry[0], ry[-1],
                                    args=(aa,bb), limit=200)[0]

$$
K=\frac{Hartree}{m_{eff}Norm^2}(\frac{1}{2a^2}+\frac{1}{2b^2}+\frac{8(a^2+b^2)}{(a+b)^4})
$$

In [13]:
eff_mass=0.277

def K11(a,b):
    Norm=np.sqrt(1+16*a**2*b**2/(a+b)**4)
    return Hartree*(1/(2*a**2)+1/(2*b**2)+8*(a**2+b**2)/(a+b)**4)/eff_mass/Norm**2

$$
V_{3}=(2\pi)^2 \int_0^\infty\int_0^\infty{\Psi (r_{1}, r_{2})^2 W_{eh}(r_{2})r_{1} r_{2}dr_{1}dr_{2}}
$$

In [14]:
def integrand33(rx,ry,a,b):
    return 27.21*rx*ry*wavefunc(rx,ry,a,b)**2*Weh(ry)

In [15]:
def V55(rx,aa,bb):    
    return 2*np.pi*integrate.quad(integrand33, ry[0], ry[-1],
                                    args=(rx,aa,bb))[0]

In [16]:
def V66(aa,bb):    
    return 2*np.pi*integrate.quad(V55, rx[0], rx[-1],
                                    args=(aa,bb))[0]

$$
H=K+V_{1}-V_{2}+V_{3}
$$

In [17]:
x=np.zeros(2)

def gs(x):
    return K11(x[0],x[1])+V22(x[0],x[1])-V44(x[0],x[1])+V66(x[0],x[1])

In [20]:
from scipy.optimize import minimize

value_min = minimize(gs, [10, 25],bounds=[(5,30),(20,50)],method='L-BFGS-B',options={'disp':1})

value_min

      fun: 0.0742226123194197
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00363836, -0.00078588])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 27
      nit: 8
   status: 0
  success: True
        x: array([30., 50.])